# Example: Deploying a Denoiser Model in Audacity

This notebook shows how to deploy the [Demucs](https://github.com/facebookresearch/denoiser) denoiser model of [Defossez et al. (2020)](https://arxiv.org/abs/2006.12847) in Audacity. Minor modifications have been made to the original code to support [JIT-scripting](https://pytorch.org/docs/stable/jit.html), which is required for `audacitorch` models.

In [1]:
import os
import json
import math
from pathlib import Path
from typing import Union
from IPython.display import Audio, display

import torch
import torchaudio

# Disable gradient computation
torch.set_grad_enabled(False)

# Constants
SAMPLE_RATE = 16000

### Create Noisy Audio Example

In [2]:
# Load audio
x, sr = torchaudio.load("../assets/audio/sample_1.wav")

# Resample to model rate
x = torchaudio.transforms.Resample(sr, SAMPLE_RATE, dtype=x.dtype)(x)

# Add noise
x += torch.randn_like(x) * 0.01

# Play audio
Audio(x.flatten().numpy(), rate=SAMPLE_RATE)

### Initialize Model & Load Weights

In [3]:
from denoiser import Denoiser

model = Denoiser()
state_dict = torch.load("../assets/models/denoiser.pt")  # Pretrained weights
model.load_state_dict(state_dict)

<All keys matched successfully>

### Compare Noisy & Processed Audio

To test the model, we attempt to remove the noise from our example recording. Note that the model operates at 16kHz, meaning some of the original high-frequency information is lost in the de-noising process.

In [4]:
# Noisy recording
Audio(x.flatten().numpy(), rate=SAMPLE_RATE)

In [5]:
# De-noised recording
Audio(model(x).flatten().numpy(), rate=SAMPLE_RATE)

### Model Metadata

To deploy our scripted model to Audacity via `audacitorch`, we need to provided the [required metadata](https://github.com/audacitorch/audacitorch#model-metadata).

In [6]:
# Create a dictionary with model metadata
metadata = {
    'displayname': 'Demucs denoiser',
    'author': 'Alexandre Defossez, Gabriel Synnaeve, and Yossi Adi',
    'sample_rate': 16000, # Demucs native sample rate; all input audio will be re-sampled to this rate
    'domain_tags': ['speech'],
    'short_description': 'Use me for speech enhancement',  # Max 60 characters
    'long_description':  'Use me for speech enhancement; I operate at 16kHz',  # Max 280 characters
    'tags': ['speech enhancement', 'speech'],
    'labels': ['enhanced'],  # Name of the output track produced by Audacity after processing
    'effect_type': 'waveform-to-waveform',  # Must be one of 'waveform-to-waveform' or 'waveform-to-labels'
    'multichannel': False, # Demucs operates on mono audio
}

### Model Wrapper & Serialization

Next, we wrap our model in a subclass of `audacitorch.core.WaveformToWaveformBase` and serialize. This will create a folder (`../outputs/denoiser/`) holding our model and metadata.

In [7]:
from audacitorch.core import WaveformToWaveformBase
from audacitorch.utils import save_model, validate_metadata

class DenoiserWrapper(WaveformToWaveformBase):
    
    # We only need to override the `do_forward_pass` method to process audio
    def do_forward_pass(self, x: torch.Tensor) -> torch.Tensor:
        
        return self.model(x)[0]  # Discard batch dimension

In [8]:
wrapped = DenoiserWrapper(model)

# Because our model and utility code are JIT-compatible, we can script with a single call
scripted = torch.jit.script(wrapped)

In [9]:
# Check that our metadata is formatted correctly
valid, msg = validate_metadata(metadata)
assert valid

# We can now save our model and metadata to disk
save_model(scripted, metadata, Path('../outputs/denoiser'))

### Add ReadMe for Model Registry

To allow Audacity to locate our model, we need to add a `README.md` file with a special tag.

In [10]:
content = '''---
tags:
- audacity
---'''

with open(Path('../outputs/denoiser') / 'README.md', 'w') as file:
    file.write(content)

### Upload to HuggingFace

Finally, we will create a HuggingFace repository with the contents of our output directory. __Fill in your HuggingFace username and access token below.__ For instructions on how to generate a HuggingFace access token, see [here](https://huggingface.co/docs/hub/security-tokens).

In [11]:
from huggingface_hub import login, create_repo, HfApi, HfFolder

In [12]:
your_name = <YOUR_USERNAME_HERE>
your_token = <YOUR_ACCESS_TOKEN_HERE>

SyntaxError: invalid syntax (2672098791.py, line 1)

In [ ]:
login(your_token)

In [ ]:
api = HfApi()

# Create a new repository
repo_url = api.create_repo(
    repo_id=f"{your_name}-denoiser-example", 
    token=your_token, 
    exist_ok=True,
    private=False,  # The repo must be public for Audacity to locate your model
    repo_type="model"
)

# Upload our output directory
api.upload_folder(
    folder_path=Path('../outputs/denoiser'),
    repo_id=f"{your_name}/{your_name}-denoiser-example",
    repo_type="model",
)

Your model should now be searchable from within Audacity's `Deep Learning Effect` menu!